<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFE_numba_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
!pip install memory_profiler

CPU times: user 448 ms, sys: 77.2 ms, total: 525 ms
Wall time: 1min 27s


In [3]:
# Load the Drive helper and mount
from google.colab import drive
%tensorflow_version 2.x
# This will prompt for authorization.
drive.mount('/content/drive')

TensorFlow 2.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%%time
import datetime
import glob
from math import sqrt
import os
import matplotlib.pyplot as plt
import bokeh
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import distance
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import Point
import networkx as nx
from sys import getsizeof
from numba import jit
import dask.dataframe as dd
import dask.array as da
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar
%matplotlib inline
%load_ext memory_profiler

pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 810 ms, sys: 130 ms, total: 940 ms
Wall time: 2.77 s


In [5]:
client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/121/1,Cluster Workers: 1 Cores: 2 Memory: 13.66 GB


Set up the path  to data and output

In [6]:
path = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input'
path_output = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output'

!ls "/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input"
!ls '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output/dask_edge_list'

buildings_raw.cpg      buildings_raw_pts.mshp  buildings_raw.shp
buildings_raw.dbf      buildings_raw_pts.prj   buildings_raw.shx
buildings_raw.prj      buildings_raw_pts.shp   GD_wind.csv
buildings_raw_pts.cpg  buildings_raw_pts.shx   source_target.csv
buildings_raw_pts.dbf  buildings_raw.qpj
edge_data.parquet


Create the functions to be used by the algorithm

In [7]:
EDGES = dd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
print(EDGES)

/usr/local/lib/python3.6/dist-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


Dask DataFrame Structure:
              source target distance  bearing IgnProb_bl
npartitions=1                                           
               int64  int64  float64  float64    float64
                 ...    ...      ...      ...        ...
Dask Name: read-parquet, 1 tasks


In [0]:
def output_df_as_arrays(edges):
  source = edges.source
  target = edges.target
  distance = edges.distance
  bearing = edges.bearing
  ign_probability = edges.IgnProb_bl
  return source, target, distance, bearing, ign_probability

In [9]:
%%time
%memit
data_source, data_target, data_distance, data_bearing, data_ign_probability = output_df_as_arrays(EDGES)

peak memory: 291.93 MiB, increment: 0.21 MiB
CPU times: user 87.5 ms, sys: 41.9 ms, total: 129 ms
Wall time: 266 ms


In [0]:
# wind scenario
@jit
def wind_scenario(file_name):
    # wind scenario conditions
    wind_data = pd.read_csv(os.path.join(path, file_name))
    i = np.random.randint(0, wind_data.shape[0])
    w = wind_data.iloc[i, 2]
    d = wind_data.iloc[i, 1]
    b = wind_data.iloc[i, 3]
    # wind direction
    wind_bearing_max = b + 45
    wind_bearing_min = b - 45
    if b == 360:
        wind_bearing_max = 45
    if b <= 0:  # should not be necessary
        wind_bearing_min = 0
    if b == 999:
        wind_bearing_max = 999
        wind_bearing_min = 0
    
    return wind_bearing_max, wind_bearing_min, d

In [0]:
# create ignitions
@jit
def ignitions(source, ign_probability):
  source_list = []

  rng = np.random.random()

  for idx, p in enumerate(ign_probability):
      if ign_probability[idx] < rng:
        source_list.append(source[idx])

  return source_list

In [0]:
# conditions of fire propagation
@jit
def conditional_spread(fires,already_burnt,
                      edges_sources,edges_targets,edges_distances, edges_bearings,
                      wind_bearing_max,wind_bearing_min,wind_distance):
  new_fires = []
  for (s,t,d,b) in zip(edges_sources,edges_targets,edges_distances,edges_bearings): 
    if s in fires and t not in already_burnt:
        if d < wind_distance:
            if b < wind_bearing_max and b > wind_bearing_min:
                new_fires.append(t)

  already_burnt.extend(fires)
  
  return new_fires, already_burnt

In [0]:
# burned assets
@jit
def burnt_assets(already_burn, fires):
  return already_burn.append(fires)

In [0]:
@jit
def main(number_of_scenarios, EDGES):
  DATA_sources, DATA_targets, DATA_distances, DATA_bearings, DATA_ign_probability = output_df_as_arrays(EDGES)
  burn_list = []
  scenario_list = []
  step_list = []
  # --- SCENARIOS
  print("number of scenarios : {}".format(number_of_scenarios))
  for scenario in range(number_of_scenarios):
      BURNED = []
      print("--- SCENARIO : {}".format(scenario))
      FIRES = ignitions(DATA_sources, DATA_ign_probability)
      if len(FIRES) == 0:
          print("no fire")
          continue
      WIND_bearing_max, WIND_bearing_min, WIND_distance = wind_scenario('GD_wind.csv')
      # --------- STEPS
      for step in range(len(EDGES)):
          print("--------- STEP : {}".format(step))
          FIRES, BURNED = conditional_spread(FIRES,BURNED,
                      DATA_sources,DATA_targets,DATA_distances, DATA_bearings,
                      WIND_bearing_max,WIND_bearing_min,WIND_distance)
          if len(FIRES) == 0:
            print("no fire")
            burn_list.extend(BURNED)
            [5] * 4
            step_list.append([step]*len(burn_list))
            break
          burn_list.extend(BURNED)
      scenario_list.extend([scenario]*len(burn_list))
  return burn_list, scenario_list, step_list

In [0]:
%%time
# with ProgressBar():
# load data
# EDGES = dd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
EDGES = pd.read_parquet(os.path.join(path_output, 'dask_edge_list', 'edge_data.parquet'), engine='pyarrow')
# run main
list_BURN, list_scenarios, list_steps = main(10, EDGES)

number of scenarios : 10
--- SCENARIO : 0
--------- STEP : 0


In [0]:
%%time
# check convergence
from itertools import groupby
number_of_burns_per_scenarios = [len(list(group)) for key, group in groupby(scos)]
cumulative_number_of_burns_per_scenarios = list(np.cumsum(number_of_burns_per_scenarios))
scenario_list = list(set(scos))
scenario_list = [x+1 for x in scenario_list]

average_burn_per_scenario = [c / s for c,s in zip(cumulative_number_of_burns_per_scenarios, scenario_list)]

In [0]:
plt.plot(scenario_list, average_burn_per_scenario)